# Checkpoint 3

**Due: Friday, 2 December, 2022 at 5:00pm GMT**

Total points: 100

### Read This First
1. Use the constants provided in the cells. Do not use your own constants.

2. Wherever you see `raise NotImplementedError()`, remove that line and put your code there.

3. Put the code that produces the output for a given task in the cell indicated. You are welcome to add as many cells as you like for imports, function definitions, variables, etc. Do not alter the argument list of functions that are given to you.

4. Your notebook must run correctly when executed once from start to finish. Your notebook will be graded based on how it runs, not how it looks when you submit it. To test this, go to the *Kernel* menu and select *Restart & Run All*.

5. Once you are happy with it, clear the output by selecting *Restart & Clear Output* from the *Kernel* menu.

6. Submit through Noteable.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import time

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 14

## Problem 1: the flight of batted baseballs (50 pts)

Batted baseballs experience enough air resistance to alter their paths from a simple parabolic motion. To properly model the flight of a baseball, we must consider the drag force, which is defined as

$
\begin{align}
\large
F_{D} = -\frac{1}{2} C_{D} A \rho v^{2},
\end{align}
$

where $C_{D}$ is the drag coefficient, $A$ is the cross-sectional area of the ball, $\rho$ is the density of air, and $v$ is the velocity of the ball relative to the air. The drag force is oriented opposite to the velocity of the ball.

The system of equations describing the motion of an object experiencing the forces of gravity and air resistance is given by

$
\begin{align}
\large
\frac{dx}{dt} = v_{x}
\end{align}
$

$
\begin{align}
\large
\frac{dy}{dt} = v_{y}
\end{align}
$

$
\begin{align}
\large
\frac{dv_{x}}{dt} = \frac{F_{D_x}}{m}
\end{align}
$

$
\begin{align}
\large
\frac{dv_{y}}{dt} = \frac{F_{D_y}}{m} - g
\end{align}
$

where $F_{D_x}$ and $F_{D_y}$ are the $x$ and $y$ components of the drag force and $m$ is the mass of the ball.

The cell below defines a function describing simple projectile motion without air resistance.

In [ ]:
g = 9.80665 # m/s^2
def projectile_motion(t, f):
    """
    f0 = x  => dx/dt  = vx
    f1 = y  => dy/dt  = vy
    f2 = vx => dvx/dt = 0
    f3 = vy => dvy/dt = - g
    """
    
    vals = np.zeros_like(f)
    vals[0] = f[2]
    vals[1] = f[3]
    vals[2] = 0
    vals[3] = - g

    return vals

## Task 1: 20 pts

Compute the motion of a batted ball **experiencing air resistance** under the following conditions.

1. The initial position of the ball is x = 0 m and y = 1 m (the approximate height of a hittable pitch).
2. The initial velocity of the ball is 50 m/s at an angle of 42 degrees with respect to the ground.
3. The ball will land in the seating area beyond the field of play. Nearest to the field, the seats are at a height of 3.5 m up from the ground. The ball should be considered to have landed when it reaches this height (3.5 m).

To do this you must define a new system of equations describing the motion of the ball under gravity and air resistance. You may start with the `projectile_motion` defined function above if you like. The relevant constants are provided in the cell below. Use these in your calculation.

In the cell with `task1`, write a function that returns the time of flight in seconds and the final x displacement of the baseball in meters when it lands in the stands. This function should accept two arguments: the initial speed of the baseball in m/s and the initial angle in degrees. **Do not modify the existing argument list for the `task1` function.** In a subsequent cell, we will call your function with the initial conditions given above. Refer to the testing cell to see how the function will be called.

Your answers must be within 0.1 s and 0.1 m of the correct time and displacement, which are not given.

In [ ]:
from scipy.integrate import solve_ivp

In [ ]:
# baseballs
m = 0.145 # mass in kg
c = 23.2  # circumference in cm
r = c / 2 / np.pi
A = np.pi * (r)**2 / 10000 # m^2
Cd = 0.34

# Earth-related constants
rhoE = 1.18 # air density at sea level kg/m^3
g = 9.80665 # m/s^2

In [ ]:
def task1(v0, theta):
    
    # define this constant to use for the drag force
    w = 0.5 * Cd * rhoE * A

    # Initial launch angle (convert to radians)
    theta0 = np.radians(theta)
    
    #function describing simple projectile motion with air resistance
    def proj_mot(t, f):
        
        """
        f[0] = x  => vals[0] = dx/dt  = vx
        f[1] = vx  => vals[1] = dvx/dt = Fdx/m    
        f[2] = y => vals[2] = dy/dt = vy
        f[3] = vy => vals[3] = dvy/dt = Fdy/m - g
        """
        vals = np.zeros_like(f)
        
        vals[0] = f[1]
        vals[2] = f[3]

        speed = np.hypot(vals[0], vals[2])
        
        vals[1] = -w/m * speed * f[1]
        vals[3] = -w/m * speed * f[3] - g
        
        return vals


    # initial conditions x0, v0x, y0, v0y
    ini0 = 0, v0 * np.cos(theta0), 1, v0 * np.sin(theta0)
    # Integration limits (integrate until tf unless the seating area is reached before)
    t0, tf = 0, 20

    
    def reach_seats(t, f):
    # We've hit the seats if they coordinate is 3.5 meters
        return f[2] - 3.5
    # Stop the integration when we hit the seats (target).
    reach_seats.terminal = True
    # The ball has to be moving downwards - to not stop the integration before we reach the top of the parabola 
    reach_seats.direction = -1


    solution = solve_ivp(proj_mot, (t0, tf), ini0, dense_output=True,events=(reach_seats))

    # time (points) from 0 until reaching the seats
    t = np.linspace(0, solution.t_events[0][0], 100)

    # solution for x and y coordinates (trajectory)
    sol = solution.sol(t)
    x, y = sol[0], sol[2]

    # time of flight until reaching seats
    tflight = solution.t_events[0][0]
    # total x-distance until reaching seats
    x_distance = x[-1]
    
    return tflight, x_distance

In [ ]:
vi = 50 # m/s
theta = 42 # degrees

tfinal1, xfinal1 = task1(vi, theta)
print (f"Flight time: {tfinal1} s.")
print (f"Final x displacement: {xfinal1} m.")

# We will test against the correct answer.


## Task 2: 20 pts

Can you hit a ball that will land in the same spot in half the time?

In the cell below with `task2`, write a function that computes the initial speed and angle required for a ball to land in the same spot as in Task 1. This function should accept a single argument, the desired flight time in seconds. **Do not modify the argument list of `task2`.** The function should return the initial speed in m/s and initial angle in degrees. Refer to the testing cell below to see how the function will be called and tested. Your answer should result in a flight time that is half of the flight time from task 1 to within 1%.

In [ ]:
from scipy.integrate import solve_bvp

In [ ]:
def task2(tflight):
    
    # define this constant to use for the drag force
    w = 0.5 * Cd * rhoE * A

    #functions describing simple projectile motion with air resistance
    def derivatives(t, f):
    
        """
        f[0] = x  => vals[0] = dx/dt  = vx
        f[1] = vx  => vals[1] = dvx/dt = Fdx/m    
        f[2] = y => vals[2] = dy/dt = vy
        f[3] = vy => vals[3] = dvy/dt = Fdy/m - g
        """
        vals = np.zeros_like(f)
        
        vals[0] = f[1]
        vals[2] = f[3]

        speed = np.hypot(vals[0], vals[2])
        
        vals[1] = -w/m * speed * f[1]
        vals[3] = -w/m * speed * f[3] - g
        
        return np.vstack((vals))

    #boundary conditions
    # f0 known boundary conditions (initial)
    # fb known boundary conditions (final)
    def bon_con (f0, fb):
        
        return np.array([f0[0], fb[0] - xfinal1, f0[2] - 1, fb[2] - 3.5])
    
    # time grid
    t = np.linspace(0, tflight, 100)
    
    # initial guesses for parameters
    f = np.zeros((4, t.size))
    f[0] = 0
    f[1] = 50
    f[2] = 1
    f[3] = 10 

    sol = solve_bvp(derivatives, bon_con, t, f, verbose=2)

    # trajectory solutions in the given time grid
    solf = sol.sol(t)
    
    # initial speed 
    vi = np.sqrt((solf[1][0])**2 + (solf[3][0])**2)

    
    # returns intial speed and angle required for a ball to land in the same spot as in Task1
    return vi, np.degrees(np.arccos(solf[1][0]/vi))

In [ ]:
factor = 2
my_vi, my_angle = task2(tfinal1 / factor)
print (f"Initial velocity: {my_vi} m/s, angle: {my_angle} degrees.")

tfinal2, xfinal2 = task1(my_vi, my_angle)
print (f"Flight time: {tfinal2} s.")
print (f"Final x displacement: {xfinal2} m.")

print (f"Task1 flight time: {tfinal1} s, task2 flight time: {tfinal2} s, ratio: {tfinal1/tfinal2}")
diff = np.abs(factor - tfinal1/tfinal2)
print (f"{diff=}")
assert diff < 0.01 * factor

## Task 3: 10 pts

Make an animation showing the flight of both baseballs (i.e., x displacement on x axis and y displacement on y axis). Time it so the slower ball launches first, followed by the faster ball, and they land at the same time.

The axes function `ax.set_aspect('equal')` (for a given axes object called `ax`) can be used to make distances the same on the x and y axes.

In [ ]:
# this function is the same as task(1) but it returns the full solution from the integration
def task3a(v0, theta):
    
    
    # define this constant to use for the drag force
    w = 0.5 * Cd * rhoE * A

    # Initial launch angle (convert to radians)
    theta0 = np.radians(theta)
    
    #function describing simple projectile motion with air resistance
    def proj_mot(t, f):
        
        """
        f[0] = x  => vals[0] = dx/dt  = vx
        f[1] = vx  => vals[1] = dvx/dt = Fdx/m    
        f[2] = y => vals[2] = dy/dt = vy
        f[3] = vy => vals[3] = dvy/dt = Fdy/m - g
        """
        vals = np.zeros_like(f)
        
        vals[0] = f[1]
        vals[2] = f[3]

        speed = np.hypot(vals[0], vals[2])
        
        vals[1] = -w/m * speed * f[1]
        vals[3] = -w/m * speed * f[3] - g
        
        return vals


    # initial conditions x0, v0x, y0, v0y
    ini0 = 0, v0 * np.cos(theta0), 1, v0 * np.sin(theta0)
    # Integration limits (integrate until tf unless the seating area is reached before)
    t0, tf = 0, 20

    
    def reach_seats(t, f):
    # We've hit the seats if they coordinate is 3.5 meters
        return f[2] - 3.5
    # Stop the integration when we hit the seats (target).
    reach_seats.terminal = True
    # The ball has to be moving downwards - to not stop the integration before we reach the top of the parabola 
    reach_seats.direction = -1


    solution = solve_ivp(proj_mot, (t0, tf), ini0, dense_output=True,events=(reach_seats))

    return solution

In [ ]:
slow_ball = task3a(vi, theta)  # parameters given in Task1
fast_ball = task3a(my_vi, my_angle)   # parameters calculated in Task2

In [ ]:
from matplotlib import animation
from IPython.display import HTML

In [ ]:
fig = plt.figure()
fig.set_dpi(100)
fig.set_size_inches(7, 6.5)

# axis limits to fit the trajectories
ax = plt.axes(xlim=(0, 130), ylim=(0, 45))
ax.set_aspect('equal')
ax.set_xlabel("x_displacement [m]")
ax.set_ylabel("y_displacement [m]")

# slow ball
patch1 = plt.Circle((5, -5), 1.5, ec = "blue", fc='y')
# fast ball 
patch2 = plt.Circle((5, -5), 1.5, ec = "brown", fc='green')



def init():
    patch1.center = (0, 1)
    ax.add_patch(patch1)
    
    patch2.center = (0, 1)
    ax.add_patch(patch2)
    return patch1, patch2,

def animate(i):
    
    # i is the frame number
    x, y = patch1.center

    # time grid for ball 1
    t1 = np.linspace(0, slow_ball.t_events[0][0], 360)

    # slow ball trajectories
    x = slow_ball.sol(t1)[0]
    y = slow_ball.sol(t1)[2]

    patch1.center = (x[i], y[i])
    
    # fast ball takes half of the time to reach the target
    # don't make fast ball ove until half of the frames have already passed
    if i > 180:
        x2, y2 = patch2.center
        # time grid for fast ball
        t2 = np.linspace(0, fast_ball.t_events[0][0], 180)
        # fast ball trajectory
        x2 = fast_ball.sol(t2)[0]
        y2 = fast_ball.sol(t2)[2]
        patch2.center = (x2[i - 180], y2[i - 180])
        
    return patch1, patch2,

anim = animation.FuncAnimation(fig, animate, 
                               init_func=init, 
                               frames=360, 
                               interval=20,
                               blit=True)
HTML(anim.to_jshtml())

## Problem 2: harmonics of the square wave (25 pts)

A square wave is composed of a fundamental tone (at wavenumber $\omega=1$) and a series of harmonics at odd wavenumbers. The amplitudes of the harmonics obey the following relation:

$
\begin{align}
\large
\frac{|c_k(\omega=3,5,7,...)|}{|c_k(\omega=1)|} \propto \omega^{\alpha},
\end{align}
$

where $c_k(\omega)$ are the Fourier coefficients as a function of wavenumber of the square wave and $\alpha$ is a constant. $c_k(\omega=1)$ is the Fourier coefficient of the fundamental tone, i.e., at $\omega=1$.

In the cell below, you are given a square wave signal.

In [ ]:
from scipy import signal

tsqu = np.linspace(0, 20*np.pi, 1001)
ysqu = signal.square(tsqu)

plt.plot(tsqu, ysqu)
plt.xlabel('t')
plt.ylabel('f(t)')
plt.title('square wave')
plt.show()

## Task 1: 20 pts

In the cell below, use the square wave signal define above to calculate the value of $\alpha$ from the previous expression. Do this by taking the FFT of the signal, locating the peaks at the harmonics, and fitting a curve to them. Do this in the cell below. Your code should print out the value of $\alpha$ and be within 1% of the correct answer.

You may find the `scipy.signal.find_peaks` function useful.

In [ ]:
import scipy

In [ ]:
# fft of the signal
ck = np.fft.rfft(ysqu)

# maximum time 
max_t = np.max(tsqu)

# wavenumbers
wn = (2*np.pi/max_t)*np.arange(ck.size)

# peaks in ck
peaks = scipy.signal.find_peaks(np.abs(ck))[0]

# values of ck at peaks
vals = np.abs(ck[peaks])


# function fit where b is proportionality constant 
def fun (x, b, alpha):
    return b*x**alpha

popt, pcov = scipy.optimize.curve_fit(fun, wn[peaks], vals)

print("Value for alpha = " + str(popt[1]))


## Task 2: 5 pts

Plot the amplitude spectrum of the square wave signal vs. wavenumber, $\omega$. This should show the harmonics occuring at odd values of $\omega$, i.e., 1, 3, 5, 7... Include on the plot the locations of the harmonics, denoted by circles. Plot a curve that fits $\omega$ and amplitude of the harmonics. Plot this in log-log to better illustrate the relation between $\omega$ and amplitude. Include all appropriate labels, legends, etc.

In [ ]:
# odd values of wn
wn_odd = np.arange(1,np.max(wn),2)

x = np.linspace(0.1,50,1000)
fig, ax = plt.subplots() 
ax.loglog(wn, np.abs(ck), label = "amplitude spectrum")
ax.loglog(x, fun(x, *popt), color = "deeppink", label = "fit")
ax.scatter(wn_odd, fun(wn_odd, *popt) , ec = "green", fc = "None", label = "Location of the harmonics" )

ax.set_xlabel("Wavenumber (log)")
ax.set_ylabel("|ck| (log)")

plt.legend()
plt.show()

## Problem 3: the solar cycle (25 pts)

The number of sunspots visible on the Sun is known to have cyclic behavior. The data file, "sunspots.txt" (included with the checkpoint), contains counts of the number of sunspots per month since 1749. The data contains two columns:
1. the time in years denoting the mid-point of the month
2. the number of sunspots observed in that month

## Task 1: 10 pts

In the cell below, write a code to compute the period of the primary mode of the solar cycle (i.e., the period corresponding to the highest peak in the amplitude spectrum, excluding the peak at $k=0$). Your code should print out the value of the period in years to within 0.2 years of the correct answer (as computed with this method).

In [ ]:
# time in years denoting the mid-point of the month 
year  = np.loadtxt("sunspots.txt")[:,:1]
year = year.flatten()

# number of sunspots observed in that month 
sunspots = np.loadtxt("sunspots.txt")[:,1:]
sunspots = sunspots.flatten()

# maximum year
tmax = np.max(year)

In [ ]:
# fft of the signal
ck2 = np.fft.rfft(sunspots)

# wavenumbers
wn2 = (2*np.pi / tmax)*np.arange(ck2.size)

In [ ]:
plt.plot(wn2, np.abs(ck2)/ck2.size)

In [ ]:
plt.plot(wn2[5:40], np.abs(ck2[5:40])/ck2.size)

In [ ]:
# find the position of the peak (given range to ignore the first peak)
center = np.abs(ck2[5:40]).argmax()

# define the region to filter
region = np.abs(wn2 - wn2[center + 5]) > 0.0020
cknew = ck2.copy()
cknew[region] = 0
# get back teh first peak
cknew[0] = ck2[0]

#filtered signal
ynew = np.fft.irfft(cknew)

plt.figure()
plt.plot(year, sunspots, label="original")
plt.plot(year, ynew, label="filtered")
plt.xlabel('time [years]')
plt.ylabel('sunspots')
plt.legend()
plt.show()

In [ ]:
# peaks in filtered signals
newpeaks = scipy.signal.find_peaks(ynew)[0]

# corresponding time for those peaks
time_peaks = year[newpeaks]

period = np.average(np.diff(time_peaks))

print("Period in years: " + str(period))


## Task 2: 10 pts

Using the period calculated previously, predict the time of the next solar maximum (i.e., the next time in the future when the number of sunspots will peak). Do this by creating a filter to include only the 30 closest values on each side of the peak. Create a new signal from the filtered Fourier coefficients and locate the last maximum. Use this to predict the time of the next maximum. Your code should print the month and year of the next maximum. Your answer should be accurate to within 2 months of the correct answer (as computed by this method).

Note, this method does not correctly calculate the time of the last peak, so your prediction will differ with the real prediction.

In [ ]:
# define the region to filter (around the already defined in Task1 center)
region2 = np.abs(wn2 - wn2[center + 5]) > 0.106

cknew2 = ck2.copy()
cknew2[region2] = 0

check = len(np.abs(cknew2[cknew2 != 0]))
# check that there are 60 values (30 each side)
print(check)

 
# get back the first peak 
cknew2[0] = ck2[0]

In [ ]:
# filtered signal
ynew2 = np.fft.irfft(cknew2)

In [ ]:
# new peaks in filtered signal
newpeaks2 = scipy.signal.find_peaks(ynew2)[0]

# corresponding time for those peaks
time_peaks2 = year[newpeaks2]

next_max = time_peaks2[-1] + period
print("Next Maximum: " + str(next_max ))
print("Next Maximum: Mid April 2024")

## Task 3: 5 pts

Make a plot including the original data of the number of sunspots vs. time, your filtered signal from the previous task, and a vertical line denoting the predicted time of the next maximum. Include all appropriate labels, units, and legends.

In [ ]:
plt.figure()
plt.plot(year, sunspots, label="original", color = "yellow")
plt.plot(year, ynew2, label="filtered", color = "deeppink")
plt.xlabel('time [year]')
plt.ylabel('number of sunspots')
plt.axvline(x = next_max, color = 'green', label = 'Next maximum')

plt.legend()
plt.show()